In [78]:
import os
from datetime import datetime
import requests
import tiktoken
import json
from typing import Dict, Tuple
from dotenv import load_dotenv
load_dotenv()

model = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model(model)

True

In [79]:
headers = {
    'Content-Type': 'application/json'
}
freshdesk_api_key = os.environ.get('FRESHDESK_API_KEY')
auth = (freshdesk_api_key, 'X')
domain = 'nueca'

import requests

tickets_url = f"https://{domain}.freshdesk.com/api/v2/tickets?order_by=created_at&order_type=desc&per_page=3"
ticket_response = requests.get(tickets_url, headers=headers, auth=auth)
tickets = ticket_response.json() if ticket_response.status_code == 200 else []

In [80]:
def parse_freshdesk_conversations(conversations, ticket_description) -> dict:
    # sorted_conversations = sorted(conversations, key=lambda x: x['created_at'])
    
    customer_conversation = ""
    agent_conversation = ""

    message_data = {
        'id': ticket_description['id'],
        'body_text': ticket_description['subject'],
        'role': 'Customer'
    }

    customer_conversation += message_data['body_text'] + "\n"

    for i, conv in enumerate(conversations, 2):
        message_data = {
            'body_text': conv['body_text'],
        }
        if message_data['body_text'] == '':
            continue
        if conv['incoming']:
            customer_conversation += message_data['body_text'] + "\n"
        else:
            agent_conversation += message_data['body_text'] + "\n"

    return {
        'customer_conversation': customer_conversation,
        'agent_conversation': agent_conversation,
    }

In [81]:
def get_all_conversations(ticket_id, ticket_description):
    conversations = []
    url = f"https://{domain}.freshdesk.com/api/v2/tickets/{ticket_id}/conversations"

    while url:
        response = requests.get(url, headers=headers, auth=auth)
        if response.status_code != 200:
            break
        data = response.json()
        conversations.extend(data)
        link = response.headers.get('link')
        if link and 'rel="next"' in link:
            import re
            match = re.search(r'<([^>]+)>;\s*rel="next"', link)
            url = match.group(1) if match else None
            print("has pagination")
        else:
            url = None

    # print(conversations)
    parsed_conversations = parse_freshdesk_conversations(conversations, ticket_description)

    return parsed_conversations

In [82]:
def add_to_map(mmap: Dict[int, Tuple[int, int]], key: int, value: Tuple[int, int]):
    if key in mmap:
        existing = mmap[key]
        mmap[key] = (existing[0] + value[0], existing[1] + value[1])
    else:
        mmap[key] = value

    return mmap

In [83]:
mmap: Dict[int, Tuple[int, int]] = {}
mmap

{}

In [84]:
# 11810

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

months = set(())

for ticket_id in range(7713, 22606):
    description_url = f"https://{domain}.freshdesk.com/api/v2/tickets/{ticket_id}"
    ticket_description = requests.get(description_url, headers=headers, auth=auth).json()
    
    date_str = ticket_description['created_at']
    dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    month = dt.month

    parsed_conversations = get_all_conversations(ticket_id, ticket_description)
    
    customer_messages = parsed_conversations['customer_conversation']
    customer_length = len(customer_messages)
    
    agent_messages = parsed_conversations['agent_conversation']
    agent_length = len(agent_messages)

    mmap = add_to_map(mmap, month, (customer_length, agent_length))

    if ticket_id % 100 == 0:
        print(f'ID: {ticket_id} - {mmap}')
        print("-----")

ID: 7800 - {12: (43, 132), 1: (10153, 26336)}
-----
ID: 7900 - {12: (43, 132), 1: (20200, 50096)}
-----
ID: 8000 - {12: (43, 132), 1: (37086, 68047)}
-----
ID: 8100 - {12: (43, 132), 1: (48091, 74466)}
-----
ID: 8200 - {12: (43, 132), 1: (66583, 99802)}
-----
ID: 8300 - {12: (43, 132), 1: (81349, 125663)}
-----
has pagination
has pagination
ID: 8400 - {12: (43, 132), 1: (102057, 156239)}
-----
has pagination
ID: 8500 - {12: (43, 132), 1: (118831, 180788)}
-----
ID: 8600 - {12: (43, 132), 1: (131362, 200941)}
-----
ID: 8700 - {12: (43, 132), 1: (144017, 220459)}
-----
ID: 8800 - {12: (43, 132), 1: (157895, 242813)}
-----
ID: 8900 - {12: (43, 132), 1: (173878, 270800)}
-----
ID: 9000 - {12: (43, 132), 1: (188791, 290088)}
-----
has pagination
ID: 9100 - {12: (43, 132), 1: (205200, 316437)}
-----
has pagination
ID: 9200 - {12: (43, 132), 1: (220606, 341271)}
-----
ID: 9300 - {12: (43, 132), 1: (234385, 365790)}
-----
ID: 9400 - {12: (43, 132), 1: (252650, 399169)}
-----
has pagination
ID:

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [86]:
mmap = {1: (403353, 669959), 2: (356369, 684548), 3: (418523, 822324)}

In [7]:
11566 - 7713

3853

In [30]:
417780 + 669959 + 347650 + 575724

2011113

In [39]:
ans = 2011113 / 3853
ans

521.960290682585

In [85]:
ticket_id

13990

In [ ]:
tokens:

get problem (conversation)
    Input
        - average conversation length = (ave customer + ave agent) ()
        - problem_prompt (990 tokens)
    Output
        - problem and resolution(105 tokens)

rag classification
    Input
        - 

rag


search results 5 fetch
average length 

In [24]:
problem = "Customer reported that delivery team took items and receipt but didn't provide replacement during order delivery. The issue occurred during the delivery process when the customer tried to return items. The customer is now awaiting the replacement items with their new order."
resolution = "Agent investigated the issue by asking the customer about their interaction with the delivery team. The resolution is pending as the agent is still gathering information about the customer's return of items to the delivery team. The case will be RESOLVED once the replacement items are confirmed to be delivered with the customer's next order."

problem_tokens = len(encoding.encode(problem))
resolution_tokens = len(encoding.encode(resolution))

print((problem_tokens + resolution_tokens) * 5)

525


In [23]:
problem = "Customer's account verification process was delayed, preventing order placement for their store. The account verification was pending due to the absence of an agent visit. The customer was unable to place orders through their own account."
resolution = "The agent followed up with the assigned agent to verify the customer's store, and the verification was eventually completed after multiple follow-ups. The customer's account was verified after the agent visited the store, allowing the customer to place orders. The issue was RESOLVED after the agent's visit and subsequent verification."

problem_tokens = len(encoding.encode(problem))
resolution_tokens = len(encoding.encode(resolution))

print(problem_tokens)
print(resolution_tokens)

41
59


'created_at': '2025-07-03T00:55:24Z'


In [25]:
prompt_template = """
## CONTEXT
You are analyzing customer support conversations for Tindahang Tapat, a digital platform enabling sari-sari stores to order groceries via mobile phone. Your task is to extract standardized problem-resolution pairs for a RAG knowledge base.

## ANALYSIS REQUIREMENTS

### Problem Statement Extraction
Create a **searchable, generic problem description** that:
- **Focuses on function over emotion**: Describe what failed/broke, not how customers felt
- **Uses standard terminology**: Platform features, order processes, payment methods, product categories
- **Removes ALL specificity**: No customer names, order numbers, specific brands, or timestamps
- **Enables similarity matching**: Use consistent vocabulary for similar issues
- **Length**: Exactly 2-3 sentences

### Resolution Documentation  
Provide an **actionable solution summary** that:
- **Details specific steps**: What the agent did to resolve the issue
- **Includes verification**: How resolution was confirmed
- **Notes preventive measures**: Steps to avoid recurrence (if applicable)
- **States clear outcome**: "RESOLVED" with method, or "UNRESOLVED" with next steps
- **Length**: Exactly 2-3 sentences

## MANDATORY STANDARDIZATION RULES

### CRITICAL: Product Normalization
**ALWAYS replace ALL specific product names, brands, and descriptions with "items"**
- "Nescafe 3-in-1 sachets" → "items"
- "Tide powder 1kg" → "items"
- "Lucky Me instant noodles" → "items"
- "Coca-Cola 1.5L bottles" → "items"
- "Pantene shampoo 200ml" → "items"
- "instant coffee product" → "items"
- "laundry detergent" → "items"
- "beverage products" → "items"
- ANY specific product reference → "items"

### Other Critical Normalizations
**Customer References**
- Any customer name → "customer"
- Any store name/location → "customer's store"
- Any order number/ID → "customer order"
- Any account details → "customer account"
- Specific amounts → "order amount" or "payment amount"

**Technical Terms**
- "App crashed/froze/stopped/won't work" → "mobile application became unresponsive"
- "Payment failed/declined/error/won't process" → "payment processing failed"
- "Can't login/access/sign in" → "authentication failed"
- "Won't load/loading/stuck loading" → "content failed to load"
- "System error/bug/glitch" → "system malfunction occurred"

**Process References**
- "Ordering/checkout/purchasing process" → "order placement process"
- "Delivery tracking/monitoring" → "order status tracking"
- "Cart/basket/shopping list" → "shopping cart"
- "Account setup/registration" → "account creation process"

### Language Standardization
- **Payment issues**: "payment processing", "transaction failure", "payment method error"
- **Delivery problems**: "delivery scheduling issue", "logistics coordination failure", "order fulfillment problem"
- **Product concerns**: "item availability issue", "inventory discrepancy", "catalog display problem"
- **Account access**: "login authentication failure", "account verification issue", "profile access problem"
- **App functionality**: "mobile app malfunction", "platform feature failure", "system functionality error"

### Absolute Exclusions
**NEVER include:**
- Any brand names, specific product names, or detailed item descriptions
- Customer names, store names, specific locations
- Order numbers, account IDs, reference codes, transaction IDs
- Agent names, department names, company structure references
- Timestamps, dates, or time-specific information
- Monetary amounts (use "order amount" instead)
- Emotional language ("frustrated", "angry", "happy", "satisfied")
- Greetings, sign-offs, pleasantries, or social conversation
- Repetitive confirmations or multiple explanations of same issue

## CONVERSATION DATA
{conversation}

## CRITICAL INSTRUCTIONS
1. Read the conversation completely
2. Identify the PRIMARY functional problem and its resolution
3. Apply ALL standardization rules - especially product normalization to "items"
4. Write exactly 2-3 sentences for each section
5. Verify NO specific product names, brands, or identifiers remain
6. Ensure language is generic and searchable

## OUTPUT REQUIREMENTS
Provide ONLY the sections below. No explanations, commentary, or additional text.

CORE PROBLEM STATEMENT:
[Generic, searchable problem description with ALL products normalized as "items"]

RESOLUTION:
[Actionable solution steps and final status: RESOLVED or UNRESOLVED with next steps]
"""
problem_prompt = prompt_template.format(conversation="")

problem_tokens = len(encoding.encode(problem_prompt))

problem_tokens

990

tickets_url = f"https://nueca.freshdesk.com/api/v2/tickets/7133"
ticket_description = requests.get(tickets_url, headers=headers, auth=auth).json()

date_str = ticket_description['created_at']
dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
month = dt.month
month

date_str